In [1]:
import math

from PIL import Image
from PIL import ImageOps

#import numpy as np
import autograd.numpy as np

def calculate_edges(img, size):
    row_coord, col_coord = np.nonzero(-(np.matrix(img.getdata()).reshape(size)) + 255)
    # Left, Top, Right, Bottom
    return np.min(col_coord) - 1, np.min(row_coord) - 1, np.max(col_coord) + 1, np.max(row_coord) + 1

def calculate_normalizing_dimensions(image, theta):
    rotated_image = ImageOps.crop(ImageOps.expand(image, 750, 255).rotate(theta), 500)
    sides = calculate_edges(rotated_image, (1000, 1000))
    return sides

def generate_sample(image, theta, normalizing_dimensions):
    """Rotates an image, downsample it to a 28x28 sample, and then reatures the 28*28 feature vector.
    
    :param - img
        The image to be transformed and downsampled.
        
    :param - theta
        The angle to transform the image before downsampling.
    """
    rotated_image = ImageOps.crop(ImageOps.expand(image, 750, 255).rotate(theta), 500)
    
    #sides = calculate_edges(rotated_image, (1000, 1000))
    
    # Image is 1000 x 1000
    # normalizing_dimensions is width x height
    #width_difference = 1000 - normalizing_dimensions[0]
    #height_difference = 1000 - normalizing_dimensions[1]
    #expansion = max(width_difference, height_difference) / 2
    #compression = min(width_difference, height_difference)
    #expansion_image = ImageOps.expand(rotated_image, expansion, 255)
    
    #if width_difference < height_difference:
    #    side_left = (height_difference - width_difference) / 2
    #    side_top = 0
    #    side_right = expansion_image.size[0] - ((height_difference - width_difference) / 2)
    #    side_bottom = expansion_image.size[1]
    #else:
    #    side_left = 0
    #    side_top = (width_difference - height_difference) / 2
    #    side_right = expansion_image.size[0]
    #    side_bottom = expansion_image.size[1] - ((width_difference - height_difference) / 2)
    
    #sides = (side_left, side_top, side_right, side_bottom)
    
    #cropped_image = rotated_image.crop(sides)

    cropped_image = rotated_image.crop(normalizing_dimensions)
    
    # Makes it a square
    cropped_max_size = max(cropped_image.size)
    
    cropped_difference_half = int(((1.0 * cropped_max_size) - min(cropped_image.size)) / 2)
    
    expanded_cropped_image = ImageOps.expand(cropped_image, cropped_difference_half, 255)
    
    expanded_cropped_image_size = expanded_cropped_image.size
    
    # Width is greater
    if cropped_image.size[0] > cropped_image.size[1]:
        cropped_expanded_cropped_image = expanded_cropped_image.crop((cropped_difference_half,
                                                                     0,
                                                                     expanded_cropped_image_size[0] - cropped_difference_half,
                                                                     expanded_cropped_image_size[1]))
    # Height is greater
    else:
        cropped_expanded_cropped_image = expanded_cropped_image.crop((0,
                                                                     cropped_difference_half,
                                                                     expanded_cropped_image_size[0],
                                                                     expanded_cropped_image_size[1] - cropped_difference_half))
    
    
    #return cropped_expanded_cropped_image
    return (-np.array(cropped_expanded_cropped_image.resize((28,28)).getdata()) + 255) * 1.0 / 255
    #img_data = np.matrix(list(rotated_img.getdata()))
    
    
    #d = c.reshape((2000, 2000))
    #e = (-d) % 255
    #e = ImageOps.crop(ImageOps.expand(a, 250, 255).rotate(45), 146)
    #math.floor((1000 - 500*math.sqrt(2)) / 2)

    #scaled_img = rotated_img.resize((128,128))
    
    #return scaled_img

example = Image.open('characters/traditional_dragon_15.png')
example2 = Image.open('characters/traditional_dragon_03.png')

z = generate_sample(example2, 90, calculate_normalizing_dimensions(example, 90))
#Image.fromarray((-(z.reshape(28,28) * 255) + 255).astype('uint8'), 'L')
#z.resize((28,28))
#z.shape
#z
#calculate_prescale_dimensions(example, 90)
#y = np.array(z.getdata())
#y.size
#z.size
#Image.fromarray(y.reshape(445,445), 'I')
#Image.frombuffer('I', (445, 445), y, 'raw', 'I', 0, 255)
#Image.fromarray(np.array(z.getdata()).astype('uint8').reshape((445,445)))

def view_feature_array(array, size):
    return Image.fromarray((-(array.reshape(size) * 255) + 255).astype('uint8'), 'L')

#view_feature_array(z, (28, 28))

#view_feature_array(np.array([generate_sample(example, 90, calculate_normalizing_dimensions(example, 90)),
#         generate_sample(example2, 90, calculate_normalizing_dimensions(example, 90))])[1],(28,28))

In [2]:
#view_feature_array(z3[1], (28,28))

name_class_list = [
    ('traditional_dragon_00.png', 0),
    ('traditional_dragon_01.png', 1),
    ('traditional_dragon_02.png', 2),
    ('traditional_dragon_03.png', 3),
    ('traditional_dragon_04.png', 4),
    ('traditional_dragon_05.png', 5),
    ('traditional_dragon_06.png', 6),
    ('traditional_dragon_07.png', 7),
    ('traditional_dragon_08.png', 8),
    ('traditional_dragon_09.png', 9),
    ('traditional_dragon_10.png', 10),
    ('traditional_dragon_11.png', 11),
    ('traditional_dragon_12.png', 12),
    ('traditional_dragon_13.png', 13),
    ('traditional_dragon_14.png', 14),
    ('traditional_dragon_15.png', 15)]
#    ('simplified_dragon_00.png', 16),
#    ('simplified_dragon_01.png', 17),
#    ('simplified_dragon_02.png', 18),
#    ('simplified_dragon_03.png', 19),
#    ('simplified_dragon_04.png', 20)]

In [3]:
image_class_list = map(lambda x: (Image.open('characters/' + x[0]), x[1]), name_class_list)

In [4]:
def generate_data_sets(image_class_pairs, test_set_size):
    train_set_size = 10 * test_set_size
    sub_image_pairs = image_class_pairs[:-1]
    main_image_pair = image_class_pairs[-1]
    
    mu = 0
    sigma = 0.1
    
    
    # Creating training data
    print 'Generating training data'
    aggregated_generated_image_list = []
    aggregated_generated_class_list = []
    
    for pair_index, pair in enumerate(image_class_pairs):
        print 'Generating images for image', (pair_index + 1), ' out of', len(image_class_pairs)
        
        sampled_rotations = np.random.normal(mu, sigma, train_set_size) * 360
        generated_image_list = []
        generated_class_list = []
        
        for index, theta in enumerate(sampled_rotations):
            normalizing_size = calculate_normalizing_dimensions(main_image_pair[0], theta)
            sample = generate_sample(pair[0],
                                     theta,
                                     normalizing_size)
            generated_image_list.append(sample)
            generated_class_list.append(pair[1])
            if 0 == (index + 1) % 10:
                print 'Generated', (index + 1), 'samples out of', train_set_size, '.'
    
        combined_generated_image_array = np.vstack(generated_image_list)
        aggregated_generated_image_list.append(combined_generated_image_array)
        
        combined_generated_class_array = np.array(generated_class_list)
        aggregated_generated_class_list.append(combined_generated_class_array)
        
    aggregated_generated_image_array = np.vstack(aggregated_generated_image_list)
    aggregated_generated_class_array = np.hstack(aggregated_generated_class_list)
    
    train_x = aggregated_generated_image_array
    train_y = aggregated_generated_class_array
    
    
    # Creating testing data
    print 'Generating testing data'
    aggregated_generated_image_list = []
    aggregated_generated_class_list = []
    
    for pair_index, pair in enumerate(image_class_pairs):
        print 'Generating images for image', (pair_index + 1), ' out of', len(image_class_pairs)
        
        sampled_rotations = np.random.normal(mu, sigma, test_set_size) * 360
        generated_image_list = []
        generated_class_list = []
        
        for index, theta in enumerate(sampled_rotations):
            normalizing_size = calculate_normalizing_dimensions(main_image_pair[0], theta)
            sample = generate_sample(pair[0],
                                     theta,
                                     normalizing_size)
            generated_image_list.append(sample)
            generated_class_list.append(pair[1])
            if 0 == (index + 1) % 10:
                print 'Generated', (index + 1), 'samples out of', train_set_size, '.'
    
        combined_generated_image_array = np.vstack(generated_image_list)
        aggregated_generated_image_list.append(combined_generated_image_array)
        
        combined_generated_class_array = np.array(generated_class_list)
        aggregated_generated_class_list.append(combined_generated_class_array)
        
    aggregated_generated_image_array = np.vstack(aggregated_generated_image_list)
    aggregated_generated_class_array = np.hstack(aggregated_generated_class_list)
    
    test_x = aggregated_generated_image_array
    test_y = aggregated_generated_class_array
    
    return train_x, train_y, test_x, test_y

tr_x, tr_y, te_x, te_y = generate_data_sets(image_class_list, 1)

Generating training data
Generating images for image 1  out of 16
Generated 10 samples out of 10 .
Generating images for image 2  out of 16
Generated 10 samples out of 10 .
Generating images for image 3  out of 16
Generated 10 samples out of 10 .
Generating images for image 4  out of 16
Generated 10 samples out of 10 .
Generating images for image 5  out of 16
Generated 10 samples out of 10 .
Generating images for image 6  out of 16
Generated 10 samples out of 10 .
Generating images for image 7  out of 16
Generated 10 samples out of 10 .
Generating images for image 8  out of 16
Generated 10 samples out of 10 .
Generating images for image 9  out of 16
Generated 10 samples out of 10 .
Generating images for image 10  out of 16
Generated 10 samples out of 10 .
Generating images for image 11  out of 16
Generated 10 samples out of 10 .
Generating images for image 12  out of 16
Generated 10 samples out of 10 .
Generating images for image 13  out of 16
Generated 10 samples out of 10 .
Generatin

In [8]:
tr_x.shape

(160, 784)

In [7]:
from autograd import grad
#import autograd.numpy as np

In [ ]:
# DO NOT RUN THIS! HERE FOR EXAMPLE ONLY!

import cPickle, gzip, numpy

# Load the dataset
f = gzip.open('mnist.pkl.gz', 'rb')
train_set, valid_set, test_set = cPickle.load(f)
f.close()

def shared_dataset(data_xy):
    """ Function that loads the dataset into shared variables

    The reason we store our dataset in shared variables is to allow
    Theano to copy it into the GPU memory (when code is run on GPU).
    Since copying data into the GPU is slow, copying a minibatch everytime
    is needed (the default behaviour if the data is not in a shared
    variable) would lead to a large decrease in performance.
    """
    data_x, data_y = data_xy
    #shared_x = theano.shared(numpy.asarray(data_x, dtype=theano.config.floatX))
    #shared_y = theano.shared(numpy.asarray(data_y, dtype=theano.config.floatX))
    # When storing data on the GPU it has to be stored as floats
    # therefore we will store the labels as ``floatX`` as well
    # (``shared_y`` does exactly that). But during our computations
    # we need them as ints (we use labels as index, and if they are
    # floats it doesn't make sense) therefore instead of returning
    # ``shared_y`` we will have to cast it to int. This little hack
    # lets us get around this issue
    #return shared_x, T.cast(shared_y, 'int32')
    return data_x, data_y

test_set_x, test_set_y = shared_dataset(test_set)
valid_set_x, valid_set_y = shared_dataset(valid_set)
train_set_x, train_set_y = shared_dataset(train_set)

batch_size = 500    # size of the minibatch

# accessing the third minibatch of the training set

data  = train_set_x[2 * batch_size: 3 * batch_size]
label = train_set_y[2 * batch_size: 3 * batch_size]

In [18]:
# Logistic Regression

def linear_activation(W, b, X):
    """
    :param - W - m input nodes (one for each feature) with c output nodes (one for each class)
    :param - b - c bias weight nodes (one for each class)
    :param - X - n samples with m features each, nxm matrix
    """
    return np.dot(X, W) + b

# P(Y = i | x, W, b)
# p_y_given_x
def softmax(W, b, X):
    """
    :param - W - m input nodes (one for each feature) with c output nodes (one for each class)
    :param - b - c bias weight nodes (one for each class)
    :param - X - n samples with m features each, nxm matrix
    """
    return np.exp(np.dot(X, W) + b) / np.sum(np.exp(np.dot(X, W) + b), axis=1)[:, None]

# y_pred
def prediction(W, b, X):
    """
    :param - W - m input nodes (one for each feature) with c output nodes (one for each class)
    :param - b - c bias weight nodes (one for each class)
    :param - X - n samples with m features each, nxm matrix
    """
    return np.argmax(softmax(W, b, X), axis=1)

def likelihood(W, b, X, Y):
    """
    :param - W - m input nodes (one for each feature) with c output nodes (one for each class)
    :param - b - c bias weight nodes (one for each class)
    :param - X - n samples with m features each, nxm matrix
    :param - Y - n classifications (one for each sample)
    """
    return softmax(W, b, X)[np.arange(Y.shape[0]), Y]

def negative_log_likelihood(W, b, X, Y):
    """
    :param - W - m input nodes (one for each feature) with c output nodes (one for each class)
    :param - b - c bias weight nodes (one for each class)
    :param - X - n samples with m features each, nxm matrix
    :param - Y - n classifications (one for each sample)
    """    
    return -likelihood(W, b, X, Y)

def loss(W, b, X, Y):
    """
    :param - W - m input nodes (one for each feature) with c output nodes (one for each class)
    :param - b - c bias weight nodes (one for each class)
    :param - X - n samples with m features each, nxm matrix
    :param - Y - n classifications (one for each sample)
    """
    return np.mean(negative_log_likelihood(W, b, X, Y))

def errors(W, b, X, Y):
    """
    :param - W - m input nodes (one for each feature) with c output nodes (one for each class)
    :param - b - c bias weight nodes (one for each class)
    :param - X - n samples with m features each, nxm matrix
    :param - Y - n classifications (one for each sample)
    """
    return np.mean(np.not_equal(prediction(W, b, X), Y))

loss_grad_W = grad(loss, 0)
loss_grad_b = grad(loss, 1)

class LogisticRegression():
    def __init__(self, n_in, n_out, learning_rate=0.13):
        self.W = np.random.rand(n_in * n_out).reshape(n_in, n_out)
        self.b = np.zeros((n_out,), dtype=np.float64)
        self.learning_rate = learning_rate

def logistic_regression_update_values(lr, X, Y):
    W_update = lr.W - lr.learning_rate * loss_grad_W(lr.W, lr.b, X, Y)
    b_update = lr.b - lr.learning_rate * loss_grad_b(lr.W, lr.b, X, Y)
    
    return W_update, b_update
        
    
def logistic_regression_training_environment(lr, data, labels, epochs=100, batch_size=500):
    n_train_batches = data.shape[0] // batch_size
    
    for epoch_count in range(epochs):
        for index in range(n_train_batches):
            X_batch = data[index * batch_size: (index + 1) * batch_size]
            Y_batch = labels[index * batch_size: (index + 1) * batch_size]
            
            lr.W, lr.b = logistic_regression_update_values(lr, X_batch, Y_batch)
            
        if 0 == epoch_count % 10:
            print 'Epoch:', epoch_count, ', Error rate:', errors(lr.W, lr.b, data, labels)
            
def logistic_regression_demo():
    from autograd import grad
    import autograd.numpy as np
    import cPickle, gzip, numpy

    # Load the dataset
    demo_f = gzip.open('mnist.pkl.gz', 'rb')
    demo_train_set, demo_valid_set, demo_test_set = cPickle.load(demo_f)
    demo_f.close()

    demo_test_set_x, demo_test_set_y = demo_test_set
    demo_valid_set_x, demo_valid_set_y = demo_valid_set
    demo_train_set_x, demo_train_set_y = demo_train_set

    demo_n_in = 28*28
    demo_n_out = 10
    demo_learning_rate = 0.13
    
    demo_c = LogisticRegression(demo_n_in, demo_n_out, demo_learning_rate)
    
    print "Baseline:", errors(c.W, c.b, demo_test_set_x, demo_test_set_y)
    
    logistic_regression_training_environment(demo_c, demo_train_set_x, demo_train_set_y, 1000)
    
    print "End of Training:", errors(c.W, c.b, demo_test_set_x, demo_test_set_y)

In [19]:
chr_in = 28*28
chr_out = len(image_class_list)
chr_learning_rate = 0.13

c = LogisticRegression(chr_in, chr_out, chr_learning_rate)

In [20]:
print 'Baseline:', errors(c.W, c.b, te_x, te_y)

Baseline: 0.9375


In [21]:
logistic_regression_training_environment(c, tr_x, tr_y, epochs=1000, batch_size=10)

Epoch: 0 , Error rate: 0.90625
Epoch: 10 , Error rate: 0.83125
Epoch: 20 , Error rate: 0.775
Epoch: 30 , Error rate: 0.70625
Epoch: 40 , Error rate: 0.65625
Epoch: 50 , Error rate: 0.6125
Epoch: 60 , Error rate: 0.58125
Epoch: 70 , Error rate: 0.575
Epoch: 80 , Error rate: 0.575
Epoch: 90 , Error rate: 0.55625
Epoch: 100 , Error rate: 0.5375
Epoch: 110 , Error rate: 0.53125
Epoch: 120 , Error rate: 0.53125
Epoch: 130 , Error rate: 0.53125
Epoch: 140 , Error rate: 0.525
Epoch: 150 , Error rate: 0.50625
Epoch: 160 , Error rate: 0.5
Epoch: 170 , Error rate: 0.49375
Epoch: 180 , Error rate: 0.49375
Epoch: 190 , Error rate: 0.49375
Epoch: 200 , Error rate: 0.49375
Epoch: 210 , Error rate: 0.49375
Epoch: 220 , Error rate: 0.49375
Epoch: 230 , Error rate: 0.4875
Epoch: 240 , Error rate: 0.48125
Epoch: 250 , Error rate: 0.48125
Epoch: 260 , Error rate: 0.46875
Epoch: 270 , Error rate: 0.4625
Epoch: 280 , Error rate: 0.4625
Epoch: 290 , Error rate: 0.45625
Epoch: 300 , Error rate: 0.45625
Epoch